In [33]:
import pandas as pd
import datetime
import os
import numpy as np

In [34]:
week_list=['27','28','29','30','31']

In [35]:
Krogger_2017_128=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 128oz Only - FY 2017 - FY 2018 YTD w.e. Aug 3 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)
Krogger_2017_64=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 64oz Only - FY 2017 - FY 2018 YTD w.e. Aug 11 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)

Krogger_2018=pd.read_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/data/Kroger - Tangy 128oz and 64oz FY 2018 YTD w.e. Sep 8 - Weekly Store w_ Kiosk.csv",dtype=str,skiprows=8)


In [8]:
def clean_df(df):
    df=df[['WEEK_NAME','UPC','SCANNED_RETAIL_DOLLARS','SCANNED_MOVEMENT','STO_NAME','STO_ZIP','STO_ADDRESS','ITM_SCN_PRC_GRP_DESC']]
    
    df['year']=df['WEEK_NAME'].apply(lambda x: x[:4])
    
    df['week']=np.where(df['year']=='2017',df['WEEK_NAME'].apply(lambda x: str(int(x[len(x)-3:len(x)-1])-1)),
                        df['WEEK_NAME'].apply(lambda x: x[len(x)-3:len(x)-1]))
    df['zip_cd']=df['STO_ZIP'].apply(lambda x: x[:5])
    df=df[df['week'].isin(week_list)]
    df=df.rename(columns={"SCANNED_RETAIL_DOLLARS":"sales","SCANNED_MOVEMENT":"units","STO_NAME":"store",
                          'STO_ADDRESS':'address',"ITM_SCN_PRC_GRP_DESC":"product"})
    df['sales']=df['sales'].astype(float)
    df['units']=df['units'].astype(float)
    return df[['store','year','week','zip_cd','address','sales','units','product']]

In [9]:
Krogger_2017_128=clean_df(Krogger_2017_128)
Krogger_2017_64=clean_df(Krogger_2017_64)
Krogger_2018=clean_df(Krogger_2018)
Krogger_2017_128=Krogger_2017_128[Krogger_2017_128['year']=='2017']
Krogger_2017_64=Krogger_2017_64[Krogger_2017_64['year']=='2017']
Krogger_2018=Krogger_2018[Krogger_2018['year']=='2018']


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [10]:
Zip_IRI_mapping=pd.read_excel("/home/jian/SunnyD/Otherinput/All_IRI_Mkt_with_Zips_0430.xlsx",dtype=str)

In [11]:
def func(x):
    y=len(list(set(x)))
    return y

R_Mkt_zips=['27705','27713','27704','27612','27526','27529','27513','27523','27613','27609','27615','27707']
Krogger_both_year=Krogger_2018.append(Krogger_2017_128).append(Krogger_2017_64)
Krogger_both_year['store_info']=Krogger_both_year['store']+"_"+Krogger_both_year['zip_cd']+"_"+Krogger_both_year['address']
Krogger_both_year['year_week']=Krogger_both_year['year']+"_"+Krogger_both_year['week']
temp_RALEIGH=Krogger_both_year[Krogger_both_year['zip_cd'].isin(R_Mkt_zips)]
temp_RALEIGH_sales_available_stores=temp_RALEIGH.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index().rename(columns={"year_week":"weeks_with_sales"})
writer=pd.ExcelWriter("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/Investigate_in_Krogger_RALEIGH_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
temp_RALEIGH_sales_available_stores.to_excel(writer,"summary_count_weeks",index=False)
temp_RALEIGH.to_excel(writer,"temp_RALEIGH",index=False)
writer.save()


# Combined Comparison

In [12]:
Krogger_both_year=Krogger_2018.append(Krogger_2017_128).append(Krogger_2017_64)
Krogger_both_year['store_info']=Krogger_both_year['store']+"_"+Krogger_both_year['zip_cd']+"_"+Krogger_both_year['address']
Krogger_both_year['year_week']=Krogger_both_year['year']+"_"+Krogger_both_year['week']
Krogger_both_year=Krogger_both_year[Krogger_both_year['sales']>0]

In [13]:
def func(x):
    y=len(list(set(x)))
    return y

In [14]:
Krogger_both_year_existing=Krogger_both_year.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing=Krogger_both_year_existing[Krogger_both_year_existing['year_week']==9]['store_info'].unique().tolist()
Krogger_both_year=Krogger_both_year[Krogger_both_year['store_info'].isin(Krogger_both_year_existing)]

In [15]:
Krogger_both_year_compare=Krogger_both_year[Krogger_both_year['week'].isin(['27','28','29','30'])]
Krogger_both_year_compare=pd.merge(Krogger_both_year_compare,Zip_IRI_mapping,on='zip_cd',how="left")


In [16]:
Krogger_both_year_compare[(Krogger_both_year_compare['year']=='2018')]['week'].unique()

array(['27', '28', '29', '30'], dtype=object)

In [17]:
Markets_18=['BIRMINGHAM/MONTGOMERY','NEW ORLEANS/MOBILE','PHOENIX/TUCSON','LOS ANGELES','DENVER','MIAMI/FT LAUDERDALE',
            'ATLANTA','SOUTH CAROLINA','CHICAGO','BOSTON','BALTIMORE/WASHINGTON','DETROIT','MISSISSIPPI','RALEIGH/GREENSBORO',
            'NEW YORK','WEST TEX/NEW MEXICO','HOUSTON','DALLAS/FT WORTH']

def In_18(x):
    if x in Markets_18:
        y="In_18_Mkts"
    else:
        y="Other"
    return y
Krogger_both_year_compare['18_Mkt_Ind']=Krogger_both_year_compare['IRI Market'].apply(lambda x: In_18(x))

In [18]:
df_YoY_compare_summary=Krogger_both_year_compare.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()
df_YoY_compare_summary.to_csv("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/SunnyD_Krogger_4_weeks_YoY_"+str(datetime.datetime.now().date())+".csv",index=False)

# 64, 128 respectively comparison

In [19]:
Krogger_both_year_64=Krogger_both_year[Krogger_both_year['product']=="Sunny D 64 FO"]
Krogger_both_year_existing_64=Krogger_both_year_64.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing_64=Krogger_both_year_existing_64[Krogger_both_year_existing_64['year_week']==9]['store_info'].unique().tolist()
Krogger_both_year_64=Krogger_both_year_64[Krogger_both_year_64['store_info'].isin(Krogger_both_year_existing_64)]
Krogger_both_year_compare_64=Krogger_both_year_64[Krogger_both_year_64['week'].isin(['27','28','29','30'])]
Krogger_both_year_compare_64=pd.merge(Krogger_both_year_compare_64,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_64['18_Mkt_Ind']=Krogger_both_year_compare_64['IRI Market'].apply(lambda x: In_18(x))
df_YoY_compare_summary_64=Krogger_both_year_compare_64.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [20]:
Krogger_both_year_128=Krogger_both_year[Krogger_both_year['product']=="Sunny D 128 FO"]
Krogger_both_year_existing_128=Krogger_both_year_128.groupby(['store_info'])['year_week'].agg(func).to_frame().reset_index()
Krogger_both_year_existing_128=Krogger_both_year_existing_128[Krogger_both_year_existing_128['year_week']==9]['store_info'].unique().tolist()
Krogger_both_year_128=Krogger_both_year_128[Krogger_both_year_128['store_info'].isin(Krogger_both_year_existing_128)]
Krogger_both_year_compare_128=Krogger_both_year_128[Krogger_both_year_128['week'].isin(['27','28','29','30'])]
Krogger_both_year_compare_128=pd.merge(Krogger_both_year_compare_128,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_128['18_Mkt_Ind']=Krogger_both_year_compare_128['IRI Market'].apply(lambda x: In_18(x))
df_YoY_compare_summary_128=Krogger_both_year_compare_128.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [21]:
Krogger_both_year_compare_2_product=Krogger_both_year[(Krogger_both_year['store_info'].isin(Krogger_both_year_existing_64)) &\
                                                     (Krogger_both_year['store_info'].isin(Krogger_both_year_existing_128))]
Krogger_both_year_compare_2_product=Krogger_both_year_compare_2_product[Krogger_both_year_compare_2_product['week'].isin(['27','28','29','30'])]
Krogger_both_year_compare_2_product=pd.merge(Krogger_both_year_compare_2_product,Zip_IRI_mapping,on='zip_cd',how="left")
Krogger_both_year_compare_2_product['18_Mkt_Ind']=Krogger_both_year_compare_2_product['IRI Market'].apply(lambda x: In_18(x))
df_YoY_compare_summary_2_product=Krogger_both_year_compare_2_product.groupby(['18_Mkt_Ind','year','week'])['sales','units'].sum().reset_index()


In [32]:
Krogger_both_year_compare_2_product.shape

(0, 10)

In [22]:
writer=pd.ExcelWriter("/home/jian/SunnyD/On_Going_WM_KG/Krogger/output/SunnyD_Krogger_4_weeks_YoY_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

In [23]:
def YoY_summary_df(df):
    df_2017_sales=df[df['year']=='2017'].rename(columns={"sales":"sales_2017"})
    df_2018_sales=df[df['year']=='2018'].rename(columns={"sales":"sales_2018"})
    del df_2017_sales['units']
    del df_2018_sales['units']
    del df_2017_sales['year']
    del df_2018_sales['year']
    sales_yoy=pd.merge(df_2017_sales,df_2018_sales,on=["18_Mkt_Ind",'week'],how="left")
    
    df_2017_units=df[df['year']=='2017'].rename(columns={"units":"units_2017"})
    df_2018_units=df[df['year']=='2018'].rename(columns={"units":"units_2018"})
    del df_2017_units['sales']
    del df_2018_units['sales']
    del df_2017_units['year']
    del df_2018_units['year']
    units_yoy=pd.merge(df_2017_units,df_2018_units,on=["18_Mkt_Ind",'week'],how="left")
    
    output=pd.merge(sales_yoy,units_yoy,on=["18_Mkt_Ind",'week'],how="left")
    output['sales_yoy']=(output['sales_2018']-output['sales_2017'])/output['sales_2017']
    output['units_yoy']=(output['units_2018']-output['units_2017'])/output['units_2017']
    return output

In [27]:
df_YoY_compare_summary_128

,18_Mkt_Ind,year,week,sales,units
0,In_18_Mkts,2017,27,3.29,1.0
1,In_18_Mkts,2017,28,20.24,6.0
2,In_18_Mkts,2017,29,35.40,10.0
3,In_18_Mkts,2017,30,3.79,1.0
4,In_18_Mkts,2018,27,36.69,11.0
5,In_18_Mkts,2018,28,43.66,14.0
6,In_18_Mkts,2018,29,28.21,9.0
7,In_18_Mkts,2018,30,23.33,7.0
8,Other,2017,27,9.87,3.0
9,Other,2017,28,16.45,5.0


In [24]:
summary_64=YoY_summary_df(df_YoY_compare_summary_64)
summary_128=YoY_summary_df(df_YoY_compare_summary_128)
summary_both_respective=YoY_summary_df(df_YoY_compare_summary_2_product)
summary_both_together=YoY_summary_df(df_YoY_compare_summary)

/usr/local/lib/python3.6/site-packages/pandas/core/ops.py:798: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


TypeError: invalid type comparison

In [25]:
summary_64.to_excel(writer,"summary_64",index=False)
summary_128.to_excel(writer,"summary_128",index=False)
summary_both_respective.to_excel(writer,"summary_both_respective",index=False)
summary_both_together.to_excel(writer,"summary_both_together",index=False)

NameError: name 'summary_both_respective' is not defined

In [44]:
writer.save()